# PHILTER
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\floor}[1]{\lfloor #1 \rfloor}$

A notebook implementing the paper https://iopscience.iop.org/article/10.1088/2058-9565/abc096/meta using the Tequila library.

## Introduction

One of the promising applications of quantum computing is estimating the energies of a Hamiltonian. Using the quantum phase esimtation (QPE) algorithm to solve the molecular time-independent Schrodinger equation is an example of using quantum computing algorithms to find the energies of a hamiltonian. This approach has been tested exprimentally using photonic devices by using the iterative quantum phase esimtation (IOPE) method. 

There is a lot of interest in calculating the energy spectrum og a hamiltonian. There are various proposed methods to determine the hamiltonian spectra by the use of variational algorithms.

For example the witness-assisted variational eigenspectra solver (WAVES) combines the QPE and variational algorithms to find the excited states of a hamiltonian. To use this algorithm, an operator that accurately approximates the excitation from the ground state to the desired state is required, but such an operator may not be easy to derive. 

The orthogonally constrained variational quantum eigensolver with the unitary pair coupled cluster with generalized singles and doubles product ansatz is useful for finding low lying excited state energies. However, we need an accurate approximation of the ground state and this algorithm is only useful for low lying excited states.

We develop an algorithm that samples the set of energies for a given Hamiltonian within a target energy-interval without requiring good approximations of the target energy eigenstates. 

The ansatz will be used for a given energy interval to measure the energies of the target energy eigenstates. The algorithm is designed for cases where good approximations for the states in the target energy interval are not trival to prepare. 

Given a hamiltonian, ansatz and target energy-interval, the algorithm amplifies the amplitudes for the states in a restricted energy interval, reducing the probability for unwanted states. This is based on quantum amplitude amplification (QAA). 

The notebook will have the following sections:

<ol>
<li>
Section 1 gives an overview of the quantum phase esimtation (QPE), quantum amplitude amplification (QAA) and quantum amplitude estimation (QAE).
</li>
<li>   
Section 2 we combine these algorithms to compute energy eigenvalues within target energy-intervals.
</li>
<li>
Section 3 we implement the PHILTER and QPHILTER algorithms using Tequila.
</li>
<li>
Section 4 we provude conclusion and final thoughts.
</li>
</ol>

# Overview of the Quantum Phase Estimation, Quantum Amplitude Amplification and Quantum Amplitude Estimation

## Quantum Phase Estimation (QPE)

We will discuss the QPE algorithm very briefly. For a more detiled explaination of the algorithm please refer to the QPE notebook. 

Given a hamiltonian $H$ we want to determine the energy eigenvalues within a certain range. To do this we prepare an operator $O$ for which we suspect there is some overlap with the desired eigenstates. 

<center>
    $O\ket{0}^{\otimes n} = \sum_{j}a_{j}\ket{E_{j}}$, $a_{j} = \braket{E_{j}}{0}$
</center>

<p>
Where the resulting state is a linear combination of unknown energy eigenstates of $H$ using coefficents $a_{j}$.
</p>

<p>
We add another m-qubit register which stores the binary repesentation of a phase related to the energies of the state as follows  
</p>    

<p>
<center>
$\ket{\Psi} = QPE(H)O\ket{0}^{\otimes m + n} = \sum_{j} a_{j}\ket{E_{j}}(\sum^{2^{m}}_{x_{i}\in \{0, 1\}^{m}}\epsilon_{x_{i}}^{(j)}\ket{x_{i}})$
</center>    
</p>    

<p>
Where $x_{i} = x_{i}^{1}x_{i}^{2}\cdots x_{i}^{m}$, $x_{i}{\alpha} \in \{0, 1\}$ and $\epsilon_{x_{i}^{(j)}} \in C$ and $E_{j}$ is the binary repesentation of the energy eigenvalue up to a scaling factor.    
</p>    

<p>
The first summation is the approximated eigenstate using the Ansatz $O$. This is written as a summation of eigenvectors $\ket{E_{j}}$. The second summation stores the phase for the eigenstate, and measuring the second register will give one of the computational states $\ket{x_{i}}$.
</p>    

<p>
    The probability the mulitple states with bit-strings repesenting numbers close to $E_{j}$ is given by
</p>    

<p>
    <center>
        $\sum_{x_{i}: |E_{j}-x_{i}| \leq err} \bra{\Psi}(I \otimes \ket{x_{i}}\bra{x_{i}})\ket{\Psi} \geq $
$    
\begin{cases} 
      |a_{j}|^{2}\frac{8}{\pi^{2}}      & err = 1 \\
      |a_{j}|^{2}(1-\frac{1}{2(err-1)}) & err > 1 \\ 
\end{cases}$
    </center>    
</p>

<p>
Where $err \in \mathbb{Z}$, $x_{i} \in \{0, \cdots, 2^{m} -1 \}$ is the encoded phases in binary and $|E_{j} - x_{i}|$ is the difference between the energy eigenvalue $E_{j}$ and the measured state $x_{i}$. The best case is when we have no difference between the eigenstate and the prepared eigenstate from the ansatz $O$.
</p>

<p>
The probability that a measurement yields the best m-qubit to the energy $\ket{E_{j}}$ approximation is at least $\frac{8}{\pi^{2}}$. Where $a_{j}$ is the overlap between $O\ket{0}^{\otimes n}$ and $\ket{E_{j}}$. We will refer to the amplitudes $\epsilon_{x_{j}}^{(j)}$ as the QPE amplitudes. We assume an exact Hamiltonian but in practice an approximation must often be made using the Trotter and higher order Suzuki decompositions. What is stored in the m-qubit register is a binary repesentation of the energy up to a rescaling factor. The next goal with this state is to amplify the part that consists of states with energies within the target energy-interval and reduce all the other states that do not have the correct energies. 
</p>

## Quantum Amplitude Amplification (QAA)

<p>
We will discuss the QAA algorithm very briefly. For a more detailed explaination of the algorithm refer to the QAA notebook. 
</p>

<p>    
Given an operator $A$ acting on $N$ qubits we have the following state, which is to split the state into good and bad states
</p> 

<p>
<center>    
$\ket{\Phi} = A\ket{0}^{\otimes N} = \sqrt{1-\alpha}\ket{\Phi_{0}} + \sqrt{\alpha}\ket{\Phi_{1}}$
</center>    
</p>    

<p>
Where
</p>

<p>
<center>   
$\ket{\Phi_{0}} = \frac{1}{\sqrt{1 -\alpha}}\sum_{x \in X_{bad}}\alpha_{x}\ket{\phi_{x}}\ket{x}$, $\ket{\Phi_{1}} = \frac{1}{\sqrt{a}}\sum_{x \in X_{good}}\alpha_{x}\ket{\phi_{x}}\ket{x}$    
</center>    
</p>    

<p>
Here $\alpha_{x}$ are the complex amplitudes, $\ket{x}$ repesented using the computational basis states and $\ket{\phi}$ will be used for the energy and state register respectively in the next section. 
</p>    

<p>
Notice how $\braket{\Phi_{i}}{\Phi_{j}} = \delta_{ij}$ since the states are orthonormal.    
</p>    

<p>
The total probability that a measurement on the state $\Phi$ yields one of the good states, i.e, a part of the set $X_{good}$ is given by    
</p>    

<p>
<center>
$\alpha = |\braket{\Phi_{1}}{\Phi}|^{2} = \sum_{x \in X_{good}} |a_{x}|^{2}$
</center>    
</p>

<p>
If $\alpha << 1$ then the the probability of measuring the good state is almost 0. We expect to repeat the state preperations $O(\frac{1}{\alpha})$ times on average before a state in the set $X_{good}$ is found. The amplification process, orginally proposed in Grover's database searching quantum algorithm and later revised by Brasard, improves the scaling $O(\frac{1}{\alpha})$. This is done by repeatedly applying the following unitary operator
</p>    

<p>
<center>
    $Q(A, X) = -AS_{0}A^{-1}S_{x}$.
</center>    
</p>

<p>
The operator S_{x} inserts a negative sign on the states that belong to the set $X_{good}$ by doing the following operation
</p>    

<p>
<center>
$S_{x}\ket{\Phi} = \sqrt{1-\alpha}\ket{\Phi_{0}} - \sqrt{\alpha}ket{\Phi_{1}}$.
</center>
</p>    

<p>
S_{x} can be thought as an oracle marking the good states with a negative sign. The operator S_{0} changes the sign of the amplitude if and only if all the qubits are the in $\ket{0}$ state. The quantum amplitude amplification is a generalization of the Grover's algorithm. In Grover's algorithm, we only work with an equal super position as the intital state. In the quantum amplitude amplification, we can use any unitary operator $A$. The number of times we apply $Q(A, X)$ is given by the formula
</p>

<p>
<center>
$k = \floor{\frac{\pi}{4sin^{-1}(\sqrt{\alpha})}}$    
</center>    
</p>

<p>
where $\alpha$ is the initial success of measuring the good state thus acheving a scaling of $O(\frac{1}{\sqrt{\alpha}})$. The probability of measuring a good state on the state $Q^{k}(A, X)\ket{\Phi}$ is given by
</p>

<p>
<center>    
P$(x \in X_{good}) = max(1-\alpha, \alpha)$. 
</center>    
</p>    

<p>
This gives the lower bound of the probability of measuring a good state. Consider the cases:  $\alpha << 1$ after K iterations we get the following 
</p>

<p>
<center>
    $Q^{k}(A, x)\ket{\Phi} \approx \sum_{x \in X_{good}} \beta_{x}\ket{\phi_{x}}\ket{x}$, 
</center>        
</p>

<p>
where $\beta_{x}$ is an amplified coefficent. If $\alpha > \frac{1}{2}$ we would then get the value $k = 0$. The initial probability of success is the value $\alpha$ and in this case we do not do any amplification to increase the probability of measuring a good state.
</p>















## Quantum Amplitude Estimation (QAE)

<p>
We will now discuss the QAE algorithm.
</p>

<p>
We know that the QAA algorithm overshooting our value of $k$ will decreases the probability of measuring the good state. It is critical to get the correct $k$ value, which is derived from the intital probability of success $a$ in the previous section. The quantum amplitude estimation (QAE) algorithm is an application of the QPE algorithm where the goal is to estimate the intital success. This is done by estimating the eigenvalues of $Q(A, x)$. We want to estimate the value $a = sin^{2}(\theta_{a})$.
</p>


<p>
Remember a value of $\theta_{a} < \frac{\pi}{4} \implies a < \frac{1}{2}$ corresponds to an increase in the amplitude of the good states when using QAE tt is the value $\theta_{a}$ that we will estimate.
</p>

<p>
The eigenvalues of $Q(A, X)$ are $\pm2i\theta_{a}$ and can be estimated using the QPE. The output from the QAE algorithm is a $t$-bit approximation to $\theta_{a}$ which we denote by $\stackrel{\text{~}}{\theta_{a}}$. This gives an error in our estimate $\stackrel{\text{~}}{\alpha} = sin^{2}(\stackrel{\text{~}}{\theta_{a}})$ that is given by
</p>    

<p>
<center>
$|\alpha - \stackrel{\text{~}}{\alpha}| \leq 2\pi \text{ err}$ $\frac{\sqrt{\alpha(1-\alpha)}}{2^{t}} + $ $\text{err}^{2}$ $\frac{\pi^{2}}{2^{2t}}$
</center>    
</p>

<p>
If $\text{err} = 1$ then the probability is at least $\frac{8}{\pi^{2}}$ to measure the best bit-string to approximate $\theta_{a}$. If $\text{err} \in Z^{+}/\{1\}$ then the probability is greater than $1 - \frac{1}{2(\text{err}-1)}$.
</p>

<p>
The QAE algorithm has a scaling of $O(\frac{1}{2^{t}})$ of the estimation error. We have seen a new family of related algorithms with similar goals propsed, for example the iterative quantum amplitude estimation algorithm which does not depend on the QPE. This algorithm also uses fewer controlled gates and qubits to estimate the amplitude. Another choice is the fixed-point quantum search, which avoids overshooting the optimal $k$-value. This approach also acheives a quadratic advantage over the classical linear search. Let's go through the Qsearch algorithm, which does not do amplitude estimation for finding a solution $0(\frac{1}{\sqrt{\alpha}})$ and does not use additional registers or controlled operations.
</p>    

<p>
The Qsearch algorithm randomly picks an integer $i$ and applies $Q^{i}(A, X)$ exponentially increasing the search space. 
</p>

<p>
Let $0 \leq i < l$. Then the size of the search space is defined as $l$.
</p>

<p>
The probability of measuring a good state after $i$ iterations of $Q(A, X)$ is $sin^{2}((2i+1)\theta_{a})$. Picking an integer $i$ randomly and uniformly under the constaint $0 \leq i < l$, the average probability is given by 
</p>

<p>
    <center>
        $\sum^{l-1}_{i=0} \frac{1}{l}sin^{2}((2i+1)\theta_{\alpha}) = \frac{1}{2}-\frac{sin(4l\theta_{\alpha})}{4lsin(\theta_{\alpha})}$
    </center>    
</p>    

<p>
If $l \geq \frac{1}{sin(2\theta_{\alpha)}}$ then $\frac{sin(4l\theta_{\alpha})}{4lsin(\theta_{\alpha})} \leq \frac{1}{4}$. Define $l_{0} := \frac{1}{sin(2\theta_{\alpha})}$ as the critical stage. If $l > l_{0}$ then $P(fail) \leq \frac{3}{4}$. The expected total number of iteration need to reach the critical stage is at most
</p>

<p>
    <center>
    $\frac{1}{2}\sum^{[log_{g}l_{0}]}_{s=1}g^{s-1} < 4l_{0}$
    </center>
</p>    

Where $g= \frac{8}{7}$ is the growth factorand and since $0 \leq i < l$ is chosen uniformly at random we obtain the $\frac{1}{2}$ factor. We set $g = \frac{8}{7}$ to obtain a small constant '4'. The value $g$ must be strictly between $1$ and $\frac{4}{3}$. As $g \to 1$ or $g \to \frac{4}{3}$ the constant explodes and makes the algorithm impractical. Once the critical stage is reached, the average failure probability for each loop is $\leq \frac{3}{4}$. The expected number of iterations before success after the critical stage is reached is 

<p>
    <center>
        $\frac{1}{2}\sum^{\infty}_{u=0}(\frac{3}{4})^{u}g^{[log_{g}l_{0}]+u} < 4l_{0}$.
    </center>    
</p>    

The total runtime is less than $8l_{0}$, where $l_{0} = \frac{1}{sin(2\theta_{\alpha})}$, in units of $Q(A, X)$, with the condition the intital success probability is very small $\alpha << 1$. We see that $8l_{o} \approx 4\frac{1}{\sqrt{\alpha}} = O(\frac{1}{\sqrt{a}})$.

# Amplitude Amplification for Eigenstate Selection

<p>
In this notebook we outline two main results. First is the phase-estimation interval target energy readout (PHILTER) algorithm, and the second algorithm will be the Qsearch phase-estimation interval target energy readout (QPHILTER) algorithm.
</p>

<p>
The PHILTER algorithm amplifies energy eigenstates within a target-energy interval. This idea of amplifying eigenstates based on the QPE algorithm was introduced by Ammar Daskin in the context of principle component analysis. This paper will be about the effects of the QPE amplitudes on the amplification process. An iterative version is also outlined, the IPHILTER algorithm, but we wil not dicuss this algorithm in this notebook.
</p>

<p>
The QPHILTER algorithm obtains a better scaling compared to the PHILTER algorithm, but it has the possibility of running forever.
</p>

<p>
Let's look at the scaling of the QPE, PHILTER and QPHILTER algorithms. 
<p>    


<table>
  <tr>
    <th>Algorithm</th>
    <th>Scaling</th>
  </tr>
  <tr>
    <th>QPE</th>
    <th>$O(\frac{1}{b}^{q})$</th>
  </tr>
  <tr>
    <th>PHILTER</th>
    <th>$O(2^{log_{2}(\frac{1}{b})})$</th>
  </tr>
  <tr>
    <th>QPHILTER</th>
    <th>$O(\frac{1}{\sqrt{b}})$</th>
  </tr>
</table>

<p>
In this table the computational complexity is in units of $QPE(H)O$ for measuring an energy $E \in I$. In the table $b$ denotes the initial success probability. For example, for the QPE the scaling is the expected number of $QPE(H)O$ without the amplification algorithm. The PHILTER algorithm depends on the amplitude estimation before amplitude amplification. This scales $O(2^{t})$, where $t$ is the numbert of qubits in the first resigster of the QAE algorithm. If $b > 0.25$, then $t_{ideal} = 2$. If $b > 2^{-t}$, then $t_{ideal} = \floor{log_{2}(\frac{1}{b})}$. Since we cannot know $t_{ideal}$ because the value depends on the initially unknown success probability $b$. The pragmatic approach would be to use the largest $t$ value possible.
</p>

# PHILTER: The phase-estimation interval target energy readout algorithm

<p>
Consider $A = QPE(H)O$ and split the state into good and bad states. The good states are those that have energies within the target energy-interval $E \in I$. Let's write the good states as $\ket{\Psi_{1}}$: $E \in I$ and the bad states $\ket{\Psi_{0}}$: $E \notin I$ of $\ket{\Psi}$.
</p>    

<p>
    <center>
        $\ket{\Psi_{1}} = \frac{1}{\alpha}\sum_{E_{j} \in I} a_{j}\ket{E_{j}}(\sum^{2^{m}}_{x_{i}\in \{0, 1\}^{m}}\epsilon_{x_{j}}^{(j)}\ket{x_{i}}) \approx \frac{1}{\sqrt{\alpha}}\sum_{E_{j}\in I} a_{j}\ket{E_{j}}\ket{E_{j}^{1}E_{j}^{2}\cdots E_{j}^{m}}$
    </center>    
</p>   

<p>
Where $E_{j}^{1}E_{j}^{2}\cdots E_{j}^{m}$ is the best $m$-bit approximation for the energy eigenvalue. Note that this construction is identical to the one in QAA. The states $\ket{\Psi_{1}}$ and $\ket{\Psi_{0}}$ are orthonormal and  
</p>

<p>
<center>
$a  = |\braket{\Psi_{1}}{\Psi}|^{2} = \sum_{E_{j} \in I} |a_{j}|^{2}$    
</center>    
</p>    

<p>
gives the initial probability a measurement on $\ket{\Psi}$ gives a good state. If $m$ is large the approximation of the energy eigenvalue will become more accurate. When $a << 1$ the overlap between the prepared ansatz and the good energy eigenstates is small. This makes the probability of measuring a good state almost 0 using the QPE algorithm. We expect to repeat state preparations with $QPE(H)O$ $O(\frac{1}{\alpha})$ times on average before a state with $E \in I$ is found. We want to improve this scaling of $O(\frac{1}{\alpha})$ by amplifying the amplitudes of the good states. 
</p>    

<p>
The amplitude amplification process allows us to do this amplification on the good states. The operator $S_{d}$ is chosen such that it changes the sign of the amplitudes of the good states $E \in I$. Here $d$ denotes the bits chosen to mark the target energy-interval.     
</p>    

<p>
For example we may want energies with either $E^{(1)} = 0$ or $E^{(1)} = 1$, where $E^{(1)}$ is the most significant bit in the binary repesentation of the energy. Here $d = 0$ for the first case and $d = 1$ for the latter. This corresponds to the interval $E \geq 0.5$ or $E \leq 0.5$. The operator $S_{d}$ is then a phase applied to the first equbit of the energy register $\ket{E_{j}^{1}E_{j}^{2}\cdots E_{j}^{m}}$. If you want to measure $E^{(1)} = 0$, then we apply XZX on the first qubit, or we apply Z if we want to measure $E^{1} = 1$. This can be understood using the visual below.
</p>

<img src = "pics/S_D Gate.png">

<p>
The length of the amplified bit-string $d$ denoted $len(d)$ will alway be less than $m$ and controls the size of the fixed target energy-interval. In other words, the amplification process amplifies the $len(d)$ most significant bits in the binary repesentation of energy. 
</p>

<p>
The operator $S_{d}$ can be implemented with 2(len($d$)-1) Toffoli gates on len($d$) -1 qubits. Each Toffoli gate can be implemented using a Hadamard, phase, controlled-NOT and $\pi$/8 gates. Recall the operator $S_{0}$ changes the sign of the amplitude if and only if all qubits are in the $\ket{0}$ state. 
</p>

<p>
Unsuccessful amplification can occur due to

<ul>
<li>
Unwanted energy eigenvalues that can not be written as exactly $m$ bits will product slightly stochastic outputs, and thus may erroneously give the bit-string d.
</li>
<li>
The initial success probability is very small and comparable to the QPE amplitudes.      
</li>
</ul>
</p>

<p>
Running the QAA algorithm, the states we amplify are those states that belong to the good states. For the amplification process to be successful, the following condition must be met
</p>    

<p>
    <center>
    $\sum_{E_{j} \in I \text{,} x_{i} \in X_{d}} |a_{j}\epsilon_{x_{i}}^{(j)}|^{2} > \sum_{E_{j} \notin I \text{,} x_{i} \in X_{d}} |a_{j}\epsilon_{x_{i}}^{(j)}|^{2} $
    </center>
</p>

<p>
We need the good states to be more in line with $d$ than the bad states.
</p>

<img src="pics/Target Interval.png">

<p>
In the figure above, an energy eigenvalue is outside of the target interval making the state not marked under amplification. One way to solve this problem is to add more qubits to the first register, but we can not be able to determine how many qubits will be sufficent for the eigenstate to lie inside the target interval. If we accept a larger interval $I^{'} > I$ which is expanded $2^{-\tau}$ on each side, where $1 \leq \tau$ where $\tau \in R$ which  our tolerance of error. Successful amplification will need len($d$) + $s$ qubits, where
</p>

<p>
<center>
    $s = \floor{log_{2}(\frac{2}{a}) + \tau - \text{len}(a))}$
</center>
<p>
    
<p>
and $b$ is the intital probability of success. The QAA will then filter out energy eigenvalue outside the interval $I^{'}$ increased by $2^{-r}$.
</p>

<img src="pics/PHILTER Diagram.png">

<p>
The above is the circuit diagram of the PHILTER algorithm. Notice how this circuit is the same as the QAA algorithm where $Q(A, d) = -AS_{0}A^{-1}S_{d}$ and $A = QPE(H)O$ and $S_{d}$ recongnizes the target-energy-interval thereby amplifying the target energy eigenstates. The amplification process is repeated $k$ times to achieve a probability of at least max($1-b$, $b$) for a measurement to give a good state. The ansatz is given by $A\ket{0}^{\otimes m + n}$, where $m$ determines the precision of the binary repesentation of the energy and $n$ is the number of qubits to store the ansatz $0\ket{0}^{\otimes n}$.
</p>

<p>
The PHILTER algorithm as given before
</p>

<p>
Inputs:
</p>    

<p>
$H$: Hamiltonian of interest, $O$: initial state preparation, $d \in \{0, 1\}^{len(d)}$: target energy interval, $\tau \in R| \tau \geq 1$: error tolerance and $t \in Z^{+}$: $t$-bit precision of the initial amplitude. 
</p>

<p>
Output:
</p>

<p>
An estimation of an eigenvalue $E \in I^{'}$ with precision $2^{-m}$ on the energy eigenvalue.
</p>

<p>
The pseudocode for the PHILTER algorithm is given below in two phases.    
</p>    

<ol>
<li>    
Amplitude estimation.
<p>
(a) Generate an estimation of the initial success probability $b$ with precision $2^{-t}$.    
</p>
<p>
(b) $k = \floor{\frac{\pi}{4sin^{-1}(\sqrt{b})}}$ estimate of the number of times we should apply $Q(QPE(H)O, d)$.  
</p>    
</li>    
<li>    
Amplification protocol. Let $m \geq \floor{log_{2}(\frac{2}{b}) + \tau}$.
<p>   
(a) Apply $Q^{k}(QPE(H)O, d)$ on $QPE(H)O\ket{0}^{\otimes n + m}$ and measure the energy register (m qubit register).
</p> 
<p>   
(b) If the output is $E \in I^{'}$ the problem is solved.
</p> 
<p>   
(c) Otherwise, go back to step 2 (a).
</p> 
</li>    
</ol>    

# QPHILTER (Qsearch phase-estimation interval target energy readout algorithm)

<p>
The difference between the QPHILTER algorithm and PHILTER algorithm is using the Qsearch algorithm to determine the number of repetitions instead of estimating the initial probability to calculate the optimal value of repetitions in the amplitude amplification.   
</p>    

<p>
The QPHILTER algorithm is given as the following.
</p>

<p>
Inputs:
</p>    

<p>
$H$: Hamiltonian of interest, $O$: initial state preparation, $d \in \{0, 1\}^{len(d)}$: target energy interval, $\tau \in R| \tau \geq 1$: error tolerance.
</p>

<p>
Output:
</p>

<p>
An estimation of $E \in I^{'}$ with precision $2^{-m}$ on the energy eigenvalue.
</p>

<p>
The pseudocode for the PHILTER algorithm is given below in two phases.    
</p>    

<ol>
<li>    
Initailize of $l = 1$ and set the growth factor $g = \frac{8}{7}$.   
</li>
Choose an integer $k$ uniformly at random such that $0 \leq k < l$.
<li>
Apply $Q^{k}(QPE(H)O, d)$ on $QPE(H)O\ket{0}^{\otimes n +m}$ and measure the energy register ($m$-qubit register).    
</li> 
<li>
If $E \in I$ the problem is solved.
</li>    
<li>
Otherwise set $l$ to $l * g$ and go to step 2.     
</li>    
</ol>

Notice the algorithm may run forever if the condition of the good states being in the target interval is not met.

# IPHILTER

<p>
The IPHILTER algorithm was also proposed in the paper, but we do not have an implementation of this algorithm, so we will not dicuss the algorithm in this notebook. Sadly...
</p>    

# Numerical demonstations

<p>
To see a numerical demonstation of the PHILTER algorithm please see section 4 of the paper.    
</p>    

# Discussion 

<p>
To see a discussion of this algorithm please see section 5 of the paper
</p>    

# Conclusions

<p>
We have outlined a series of algorithms for a quantum computer to discover the spectra of a Hamiltonian by sampling the set or energies in a target energy-interval. An advantage of these algorithms is that we do not need good approximations of the target energy eigenstates. These algorithms can be used where good approximations for the states in the target energy-interval are hard to obtain or unknown. This algorithm can be used for any Hamiltonian, with the purpose to determine the energy eigenvalues within a target energy-interval.
</p>    

<p>
Let's implement the PHILTER and QPHILTER algorithms using the Tequila library.
</p>    

# Implementing the algorithm using tequila

In [1]:
import numpy as np
from numpy import pi
import tequila as tq
import cmath 
from typing import Any
import types
import math
import copy
import random

In [2]:
def qft_rotations(n):
    circuit = tq.gates.H(target=0)
    
    for i in range(1, n + 1):
        for qubit in reversed(range(i)):
            circuit = tq.gates.Phase(target = i, control = qubit, phi = pi/(2 ** (i - qubit))) + circuit
        circuit = tq.gates.H(target=i) + circuit
    return circuit

In [3]:
def qft_swap(circuit, n):
    
    for qubit in range(n//2):
        circuit += tq.gates.SWAP(qubit, n-qubit - 1)
    return circuit

In [4]:
def qft(n):
    
    return qft_swap(qft_rotations(n - 1), n)

In [5]:
def qft_dagger(n):
    
    return qft(n).dagger()

In [6]:
def QPE(Unitary: 'QCircuit', Ansatz: 'QCircuit', qubits_first_reg: int, molecular: bool):
    
    # Building the circuit with the first Hadamard gate
    circuit = tq.gates.H(target = 0)
    
    # Building the Hadamard gates
    for next_target in range(qubits_first_reg - 1):
        circuit = circuit + tq.gates.H(target=next_target + 1)
    
    # Attaching the ansatz circuit
    for i in range(0,len(Ansatz.gates)):
        gate = Ansatz.gates[i]
        newtargets = []
        newcontrols = []
        
        # loop through the controls of the gate, setting the correct controls of the ansatz
        for k in range(0,len(gate.target)):
            newtargets.append(int(gate.target[k] + qubits_first_reg))
        
        # loop through the targets of the gate, setting the correct targets of the ansatz
        for k in range(0,len(gate.control)):
            newcontrols.append(int(gate.control[k] + qubits_first_reg))
        
        # Set the controls and targets of the ansatz
        gate._target = tuple(newtargets)
        gate._control = tuple(newcontrols)
    
    # Combine the Hadamard gates and the ansatz circuitry
    circuit = circuit + Ansatz
    
    # Define a dictionary to map the gates in the unitary to the correct number of repetitions in the circuit
    result = {}
    
    # Loop thorugh the m qubits
    for qubit in range(0, qubits_first_reg):
        
        # Loop through the unitary gates
        for gate in range(0, len(Unitary.gates)):
            
            # Define lists to add the new targets and controls for the gate
            newtargets = []
            newcontrols = []
            
            # Deep copy the gate to modify its controls and targets
            gatey = copy.deepcopy(Unitary.gates[gate])
            
            # Add the m qubit as a control for the gate 
            newcontrols.append(qubit)
            
            # loop and append the controls from the gate
            for k in range(0, len(gatey.control)):
                newcontrols.append(int(gatey.control[k]) + qubits_first_reg)
            
            # Loop and append the targets from the gate
            for k in range(0, len(gatey.target)):
                newtargets.append(int(gatey.target[k]) + qubits_first_reg)
            
            if molecular:
                gate_circuit = gatey.map_qubits(newtargets)
                gate_circuit._control = tuple(newcontrols)
            
            else:
                gatey._target = tuple(newtargets)
                gatey._control = tuple(newcontrols)
                gate_circuit = tq.QCircuit.wrap_gate(gatey)
            
            result[str(qubit) + '-' + str(gate)] = copy.deepcopy(gate_circuit)
            
    # Variable to keep track of repetitions to add the gates in the circuit and list to hold the gates
    exponential_growth = 1
    gates = []
    
    # loop through the dictionary attaching the controlled unitaries in the correct amount/sequence
    for qubit in range(0, qubits_first_reg):
        exponential_growth = 2 ** qubit
        
        # loop through the keys of the dictionary and check if the gate is on the mth qubit
        for key in result:
            split = key.split("-")
            if int(split[0]) == qubit:
                gates.append(result[key])
        
        # Loop through the gates list and attach the gates to the circuit
        for i in range(exponential_growth):
            for gate in gates:
                circuit = circuit + gate
        
        # Reset the gates list
        gates.clear() 
    
    # Attach the inverse fourier transformation on the first m qubits
    circuit = circuit + qft_dagger(qubits_first_reg)
    
    # Take measurements and simulate on the first m qubits
    bit_reader = []
    for measured_qubit in range(qubits_first_reg):
        bit_reader.append(measured_qubit)
    
    # Simulate on the first m qubits using 100 samples (Do we change to the wavefunction??)
    simulation = tq.simulate(circuit, samples = 100, read_out_qubits=bit_reader, backend='qiskit')
    
    print('Inner QPE Simulation')
    print(simulation)
    
    # Return the simulation as the final result
    return {'Simulation' : simulation, 'Circuit' : circuit}

In [7]:
def Quantum_Amplitude_Amplification(Ansatz: 'QCircuit', S_D: 'QCircuit', K: int):
    
    Circuit = copy.deepcopy(Ansatz)
    
    qubits = Circuit.qubits
    
    if(len(qubits) == 1):
        for i in range(K):
            Circuit = Circuit + tq.gates.Z(target = qubits) + tq.gates.X(target = qubits) + tq.gates.Z(target = qubits) + tq.gates.X(target = qubits) + S_D + Ansatz.dagger() + tq.gates.X(target = qubits) + tq.gates.Z(target = qubits) + tq.gates.X(target = qubits) + Ansatz
    else:
        last_qubit = qubits[-1]
        not_all_qubits = [x for i, x in enumerate(qubits) if i != len(qubits) - 1]
        for i in range(K):
            Circuit = Circuit + tq.gates.Z(target = qubits) + tq.gates.X(target = qubits) + tq.gates.Z(target = qubits) + tq.gates.X(target = qubits) + S_D + Ansatz.dagger() + tq.gates.X(target = qubits) + tq.gates.CZ(target = last_qubit, control = not_all_qubits) + tq.gates.X(target = qubits) + Ansatz
    
    simulation = tq.simulate(Circuit, samples = 100)
    
    return {'Simulation':simulation, 'Circuit':Circuit}

In [8]:
def Helperfunction(bitstring: list):
    
    # Get the length of the bit string (assuming all the bitstring are the same length)
    length = len(bitstring[0])
    
    # The cz-target will be the qubit of length - 1
    cz_target = length-1
    
    # Set the controls to all qubits except length - 1
    cz_control = [x for i, x in enumerate(range(length)) if i != cz_target]
    
    # Set up the controlled Z gate
    cz_gate = tq.gates.CZ(target = cz_target, control = cz_control)
    
    # Intialize the S_D operator
    S_D = tq.QCircuit()
    
    # Create the S_D with all the given bit-strings as the good state
    for qubit in bitstring:
        
        # Get the target for the bit-string
        apply_x_targets = [i for i, x in enumerate(qubit) if x == '0']
        
        # Create the x gates with the given targets
        x_gates = tq.gates.X(target = apply_x_targets)
        
        # Append the x gates, controlled z gates to the S_D operator
        S_D = S_D + x_gates + cz_gate + x_gates
    
    # Return the S_D operator
    return S_D

In [9]:
def PHILTER(Unitary: 'QCircuit', Ansatz: 'QCircuit', qubits_first_reg:int, molecular:bool, t:int, target_interval: str):
    
    # Create the S_D operator to mark the good states. 
    S_D = Helperfunction([target_interval])
    
    # Get the number of qubits operating on the Unitary.
    qubits = Unitary.qubits
    
    # Run until we have states in the target interval.
    while(True):
    
        # Create the Unitary for the QAE algorithm.
        if(len(qubits) == 1):
            Circuit = tq.gates.Z(target = qubits) + tq.gates.X(target = qubits) + tq.gates.Z(target = qubits) + tq.gates.X(target = qubits) + S_D + Ansatz.dagger() + tq.gates.X(target = qubits) + tq.gates.Z(target = qubits) + tq.gates.X(target = qubits) + Ansatz
        else:
            last_qubit = qubits[-1]
            not_all_qubits = [x for i, x in enumerate(qubits) if i != len(qubits) - 1]
            Circuit = tq.gates.Z(target = qubits) + tq.gates.X(target = qubits) + tq.gates.Z(target = qubits) + tq.gates.X(target = qubits) + Unitary + tq.gates.X(target = qubits) + tq.gates.CZ(target = last_qubit, control = not_all_qubits) + tq.gates.X(target = qubits) + Unitary.dagger() + S_D 

        # Get the simulation of the unitary.
        k_select = QPE(Unitary=Circuit, Ansatz=Unitary, qubits_first_reg=t, molecular=molecular)['Simulation']

        # Calculate the value of k (number of repetitions for the QAA).
        k_value = 0

        for k,v in k_select.items():
            if k_value < k:
                k_value = k
        
        print('K value')
        print(k_value)
        
        # Create the ansatz for the QPE circuit.
        QPE_Circuit = QPE(Unitary=Unitary, Ansatz=Ansatz, qubits_first_reg=qubits_first_reg, molecular = molecular)
        
        print('QPE simulation before PHILTER')
        print(QPE_Circuit['Simulation'])
         
        # Create the PHILTER circuit.  
        PHILTER_Circuit = Quantum_Amplitude_Amplification(Ansatz=QPE_Circuit['Circuit'], S_D=S_D, K = int(k_value))
        
        # Print the simulation and see if the user is satisfied with the good states in the target interval.
        print('PHILTER Simulation')
        print(PHILTER_Circuit['Simulation'])

        target_met = input("Is this simulation sufficent y/n?")
        
        # If yes, then return the PHILTER Circuit. If not, then repeat the process.
        if target_met == 'y':
            print('Script complete running')
            return PHILTER_Circuit

In [10]:
def QPHILTER(Unitary: 'QCircuit', Ansatz: 'QCircuit', qubits_first_reg:int, molecular:bool, t:int, target_interval: str):
    
    # Set the starting and growth constants for the QSearch algorithm.
    l = 1
    g = 8/7
    
    # Run until we have states in the target interval.
    while(True):
        
        # Pick a random integer between 0 and the interval.
        K = random.randint(0, l)
        
        # Create the S_D operator to mark the good states.
        S_D = Helperfunction([target_interval])
        
        # Create the ansatz for the QPE circuit.
        QPE_circuit = QPE(Unitary=Unitary, Ansatz=Ansatz, qubits_first_reg=qubits_first_reg, molecular=molecular)
        
        # Create the ansatz for the QPE circuit.
        QPHILTER_Circuit = Quantum_Amplitude_Amplification(Ansatz=QPE_circuit, S_D=S_D, K = K)
        
        # Print the simulation and see if the user is satisfied with the good states in the target interval.
        print(QPHILTER_Circuit['Simulation'])
        
        target_met = input("Is this simulation sufficent y/n?")
        
        # If yes, then return the QPHILTER Circuit. If not, then repeat the process and update the new l value.
        if target_met == 'y':
            print('Script complete running')
            return PHILTER_Circuit
        else:
            l = k * g

In [11]:
Unitary = tq.gates.T(target = 0)
Ansatz = tq.gates.X(target = 0)

print('before result')
print(QPE(Unitary=Unitary, Ansatz=Ansatz, qubits_first_reg=3, molecular=False)['Simulation'])

result = PHILTER(Unitary=Unitary, Ansatz=Ansatz, qubits_first_reg=3, molecular = False, t = 3, target_interval = '01')

before result
Inner QPE Simulation
+100.0000|100> 
+100.0000|100> 
Inner QPE Simulation
+100.0000|000> 
K value
0
Inner QPE Simulation
+100.0000|100> 
QPE simulation before PHILTER
+100.0000|100> 
PHILTER Simulation
+100.0000|1001> 
Is this simulation sufficent y/n?y
